<a href="https://colab.research.google.com/github/Ikmalrl/Computer-Security/blob/main/Hybrid_Demo_Secure_Chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# ============================================================
# Hybrid Cryptography Demo – Secure Chat
# ============================================================
# DESIGN:
# Combines RSA (asymmetric) for key exchange and WingsDings (symmetric)
# For fast message encryption.
# ============================================================
# PURPOSE:
# Demonstrates hybrid cryptography in a secure chat context, where
# RSA encrypts a temporary session key used by the WingsDings cipher.
# ============================================================

from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
import hashlib # Corrected import
import os
import struct
import math
from typing import List

# ---------- Utility Functions ----------

def pkcs7_pad(data: bytes, block_size: int = 8) -> bytes:
    """Apply PKCS#7 padding to data."""
    pad_len = block_size - (len(data) % block_size)
    return data + bytes([pad_len] * pad_len)

def pkcs7_unpad(data: bytes) -> bytes:
    """Remove PKCS#7 padding."""
    pad_len = data[-1]
    if pad_len < 1 or pad_len > len(data):
        raise ValueError("Invalid padding")
    return data[:-pad_len]


# ---------- Subkey Generation ----------

def generate_round_keys(master_key: bytes, rounds: int = 8) -> List[int]:
    """Generate round subkeys using SHA-256(key + round_number)."""
    round_keys = []
    for i in range(rounds):
        digest = hashlib.sha256(master_key + i.to_bytes(1, 'big')).digest()
        subkey = int.from_bytes(digest[:6], 'big')  # 48-bit subkey
        round_keys.append(subkey)
    return round_keys


# ---------- Round Function ----------

def round_function(right: int, subkey: int) -> int:
    """
    Nonlinear round function:
    1. Mix with subkey
    2. Apply small substitution using nibbles
    3. Bit rotation
    """
    x = (right ^ subkey) & 0xFFFFFFFF
    sbox = [0x6, 0x4, 0xC, 0x5, 0x0, 0x7, 0x2, 0xE,
            0x1, 0xF, 0x3, 0xD, 0x8, 0xA, 0x9, 0xB]
    out = 0
    for i in range(8):  # process 8 nibbles
        nibble = (x >> (i * 4)) & 0xF
        out |= (sbox[nibble] << (i * 4))
    # rotate left 5 bits
    return ((out << 5) | (out >> (32 - 5))) & 0xFFFFFFFF


# ---------- Core Cipher Class ----------

class CyberStone:
    NAME = "CyberStone"

    def __init__(self, key: bytes):
        if len(key) < 16:
            raise ValueError("Key must be at least 16 bytes (128 bits).")
        self.master_key = key[:16]
        self.round_keys = generate_round_keys(self.master_key)

    def encrypt_block(self, block: bytes) -> bytes:
        """Encrypt a single 8-byte block."""
        L, R = struct.unpack(">II", block)
        for i in range(8):
            temp = L ^ round_function(R, self.round_keys[i])
            L, R = R, temp
        return struct.pack(">II", L, R)

    def decrypt_block(self, block: bytes) -> bytes:
        """Decrypt a single 8-byte block."""
        L, R = struct.unpack(">II", block)
        for i in reversed(range(8)):
            temp = R ^ round_function(L, self.round_keys[i])
            R, L = L, temp
        return struct.pack(">II", L, R)

    def encrypt(self, plaintext: bytes) -> bytes:
        """Encrypt arbitrary-length plaintext."""
        padded = pkcs7_pad(plaintext, 8)
        ciphertext = b''
        for i in range(0, len(padded), 8):
            block = padded[i:i+8]
            ciphertext += self.encrypt_block(block)
        return ciphertext

    def decrypt(self, ciphertext: bytes) -> bytes:
        """Decrypt arbitrary-length ciphertext."""
        decrypted = b''
        for i in range(0, len(ciphertext), 8):
            block = ciphertext[i:i+8]
            decrypted += self.decrypt_block(block)
        return pkcs7_unpad(decrypted)


# ---------- Helper Functions ----------

def generate_rsa_keys():
    """Generate RSA public/private key pair (2048-bit)."""
    key = RSA.generate(2048)
    private_key = key
    public_key = key.publickey()
    return private_key, public_key


def rsa_encrypt(public_key, message_bytes):
    """Encrypt data using RSA public key and OAEP padding."""
    cipher_rsa = PKCS1_OAEP.new(public_key)
    return cipher_rsa.encrypt(message_bytes)


def rsa_decrypt(private_key, ciphertext):
    """Decrypt data using RSA private key."""
    cipher_rsa = PKCS1_OAEP.new(private_key)
    return cipher_rsa.decrypt(ciphertext)


# ---------- Hybrid Encryption Functions ----------

def hybrid_encrypt(message: str, rsa_pubkey: RSA.RsaKey) -> tuple:
    """
    1. Generate random symmetric session key.
    2. Encrypt message with WingsDings using session key.
    3. Encrypt session key with RSA public key.
    4. Return both ciphertexts.
    """
    # Generate 128-bit (16-byte) random session key
    session_key = os.urandom(16)

    # Encrypt message with WingsDings (symmetric)
    cipher = CyberStone(session_key)
    encrypted_message = cipher.encrypt(message.encode())

    # Encrypt session key with RSA (asymmetric)
    encrypted_session_key = rsa_encrypt(rsa_pubkey, session_key)

    return encrypted_session_key, encrypted_message


def hybrid_decrypt(enc_session_key: bytes, enc_message: bytes, rsa_privkey: RSA.RsaKey) -> str:
    """
    1. Decrypt session key with RSA private key.
    2. Decrypt message with WingsDings using session key.
    3. Return plaintext message.
    """
    # Decrypt session key with RSA
    session_key = rsa_decrypt(rsa_privkey, enc_session_key)

    # Decrypt message with WingsDings
    cipher = CyberStone(session_key)
    decrypted_message = cipher.decrypt(enc_message)

    return decrypted_message.decode(errors='ignore')


# ---------- Demo Section ----------

if __name__ == "__main__":
    print("\n===== Hybrid Cryptography Demo =====")

    # Step 1: RSA key generation (simulate user Bob having keypair)
    print("[1] Generating RSA key pair for Bob...")
    bob_private_key, bob_public_key = generate_rsa_keys()

    # Step 2: Alice prepares message and encrypts
    message = input("\n[2] Alice enters message to send securely: ")
    enc_session_key, enc_message = hybrid_encrypt(message, bob_public_key)

    print("\nEncryption complete!")
    print("Encrypted session key length:", len(enc_session_key), "bytes")
    print("Encrypted message (hex):", enc_message.hex())

    # Step 3: Bob decrypts using his private key
    print("\n[3] Bob decrypts the incoming message...")
    decrypted_message = hybrid_decrypt(enc_session_key, enc_message, bob_private_key)

    print("\nDecryption complete!")
    print("Decrypted message:", decrypted_message)

    # Step 4: Verify
    if decrypted_message == message:
        print("\nSuccess: Message decrypted correctly.")
    else:
        print("\nError: Decryption mismatch.")


===== Hybrid Cryptography Demo =====
[1] Generating RSA key pair for Bob...

[2] Alice enters message to send securely: Hi Bob!

Encryption complete!
Encrypted session key length: 256 bytes
Encrypted message (hex): e2c84ce0e5e76f59

[3] Bob decrypts the incoming message...

Decryption complete!
Decrypted message: Hi Bob!

Success: Message decrypted correctly.


In [ ]:
!pip install pycryptodome